In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import json

%matplotlib inline

# Analysis of results

## Import of data

In [2]:
DATE = "2020-12-17"


def import_native_and_bytecode(filename):
    native = []
    bytecode = []
    with open(filename) as f:
        for line in f:
            entry = json.loads(line)
            name_sections = list(entry['name'].split('.'))
            if name_sections[0].endswith('bytecode'):
                name_sections[0] = name_sections[0][:-9]
                entry['name'] = '.'.join(name_sections)
                bytecode.append(entry)
            else:
                native.append(entry)
    return pd.json_normalize(native), pd.json_normalize(bytecode)


def import_just_bytecode(filename):
    data = []
    for line in open(filename):
        entry = json.loads(line)
        name_sections = list(entry['name'].split('.'))
        name_sections[0] = name_sections[0][:-9]
        entry['name'] = '.'.join(name_sections)
        data.append(entry)
    return pd.json_normalize(data)
            
    
native, stock = import_native_and_bytecode(f'{DATE}/4.11.1+stock.orun.bench')
compile_only = import_just_bytecode(f'{DATE}/4.11.1+jit+compileonly.orun.bench')
jit_disabled = import_just_bytecode(f'{DATE}/4.11.1+jit+disabled.orun.bench')
jit = import_just_bytecode(f'{DATE}/4.11.1+jit.orun.bench')

In [3]:
compile_only

,name,command,time_secs,user_time_secs,sys_time_secs,maxrss_kB,codesize,ocaml_url,ocaml.version,ocaml.c_compiler,...,gc.allocated_words,gc.minor_words,gc.promoted_words,gc.major_words,gc.minor_collections,gc.major_collections,gc.heap_words,gc.heap_chunks,gc.top_heap_words,gc.compactions
0,naive-multilayer.,taskset --cpu-list 5 ./naive_multilayer.bc,19.329041,19.315840,0.006664,7988,0.0,../../src,4.11.1+dev_jit,gcc,...,3908428073,3908412655,3278991,3294409,15001,184,126976,1,126976,0
1,imrin_mem_rw.10_000_50_000_20_100_000_000,taskset --cpu-list 5 ./irmin_mem_rw.bc 10_000 ...,29.669523,29.657885,0.003331,25172,0.0,../../src,4.11.1+dev_jit,gcc,...,4761174129,4761149266,792477,817340,18166,8,661504,9,661504,0
2,nbody.50_000_000,taskset --cpu-list 5 ./nbody.bc 50_000_000,129.300293,129.264082,0.000000,7948,0.0,../../src,4.11.1+dev_jit,gcc,...,53000009642,53000005923,1715362,1719081,202385,415,61440,1,188416,2
3,durand-kerner-aberth.,taskset --cpu-list 5 ./durand_kerner_aberth.bc,1.352910,1.352269,0.000000,7576,0.0,../../src,4.11.1+dev_jit,gcc,...,589329228,589326695,319191,321724,2287,79,61440,1,188416,2
4,fannkuchredux.12,taskset --cpu-list 5 ./fannkuchredux.bc 12,871.174997,870.966875,0.000000,7832,0.0,../../src,4.11.1+dev_jit,gcc,...,10391898334,10391894744,83377,86967,39652,23,61440,1,188416,2
5,qr-decomposition.,taskset --cpu-list 5 ./qr_decomposition.bc,9.750122,9.743647,0.003329,9112,0.0,../../src,4.11.1+dev_jit,gcc,...,2298525753,2298516665,2833275,2842363,8788,43,311296,4,311296,0
6,spectralnorm2.5_500,taskset --cpu-list 5 ./spectralnorm2.bc 5_500,62.332654,62.311149,0.003333,8320,0.0,../../src,4.11.1+dev_jit,gcc,...,12100657183,12100532563,373948,498568,46210,120,126976,1,126976,0
7,fasta3.25_000_000,taskset --cpu-list 5 ./fasta3.bc 25_000_000,20.710190,20.700800,0.003333,7616,0.0,../../src,4.11.1+dev_jit,gcc,...,2846899927,2846896149,29545,33323,10863,8,61440,1,188416,2
8,regexredux2.,taskset --cpu-list 5 ./regexredux2.bc,10.991246,10.714222,0.273204,1217432,0.0,../../src,4.11.1+dev_jit,gcc,...,288711833,216267142,167966807,240411498,848,25,151633920,19,151633920,0
9,kb_no_exc.,taskset --cpu-list 5 ./kb_no_exc.bc,9.323424,9.306890,0.013330,10900,0.0,../../src,4.11.1+dev_jit,gcc,...,179068763,179066013,24031137,24033887,763,229,494592,8,568832,15


In [4]:
jit.sort_values(by='time_secs', ascending=False)

,name,command,time_secs,user_time_secs,sys_time_secs,maxrss_kB,codesize,ocaml_url,ocaml.version,ocaml.c_compiler,...,gc.allocated_words,gc.minor_words,gc.promoted_words,gc.major_words,gc.minor_collections,gc.major_collections,gc.heap_words,gc.heap_chunks,gc.top_heap_words,gc.compactions
4,fannkuchredux.12,taskset --cpu-list 5 ./fannkuchredux.bc 12,379.488397,379.388943,0.000000,8000,0.0,../../src,4.11.1+dev_jit,gcc,...,10391898334,10391894744,83377,86967,39652,23,61440,1,188416,2
36,fannkuchredux2.12,taskset --cpu-list 5 ./fannkuchredux2.bc 12,328.105117,328.043837,0.003333,7420,0.0,../../src,4.11.1+dev_jit,gcc,...,8192,4602,0,3590,0,0,126976,1,126976,0
33,mandelbrot6.16_000,taskset --cpu-list 5 ./mandelbrot6.bc 16_000,207.662356,207.597638,0.006664,8004,0.0,../../src,4.11.1+dev_jit,gcc,...,103041862887,103041859280,8824476,8828083,394122,2099,61440,1,188416,2
26,evolutionary_algorithm.10000_10000,taskset --cpu-list 5 ./evolutionary_algorithm....,145.109579,144.635133,0.429844,2812704,0.0,../../src,4.11.1+dev_jit,gcc,...,17030891906,15730629765,1186201,1301448342,60030,31,357150208,54,357150208,0
2,nbody.50_000_000,taskset --cpu-list 5 ./nbody.bc 50_000_000,88.224394,88.199872,0.000000,8036,0.0,../../src,4.11.1+dev_jit,gcc,...,53000009642,53000005923,1715362,1719081,202385,415,61440,1,188416,2
31,knucleotide3.,taskset --cpu-list 5 ./knucleotide3.bc,74.114721,74.009168,0.086635,272924,0.0,../../src,4.11.1+dev_jit,gcc,...,104935419,72532419,1678748,34081748,282,5,57502208,3,57502208,0
10,knucleotide.,taskset --cpu-list 5 ./knucleotide.bc,70.959650,70.852271,0.086632,408892,0.0,../../src,4.11.1+dev_jit,gcc,...,1891491094,1842832767,2684306,51342633,7036,12,87480320,11,87480320,0
6,spectralnorm2.5_500,taskset --cpu-list 5 ./spectralnorm2.bc 5_500,33.674721,33.654935,0.009996,8384,0.0,../../src,4.11.1+dev_jit,gcc,...,12100657183,12100532563,373948,498568,46210,120,126976,1,126976,0
19,lexifi-g2pp.,taskset --cpu-list 5 ./main.bc,33.095634,33.078963,0.003333,7856,0.0,../../src,4.11.1+dev_jit,gcc,...,14497763104,14497757847,6104446,6109703,55392,175,126976,1,126976,0
25,matrix_multiplication.1024,taskset --cpu-list 5 ./matrix_multiplication.b...,29.249789,29.232033,0.009997,30992,0.0,../../src,4.11.1+dev_jit,gcc,...,5255381,2102795,1537,3154123,16,5,3545600,21,3545600,0


In [8]:
def add_suffix(df, suffix):
    return df.add_suffix(suffix).rename(columns={f'name{suffix}': 'name'})

combined = (
    add_suffix(native, '_native')
    .merge(add_suffix(stock, '_stock'), on='name')
#    .merge(add_suffix(jit_disabled, '_jit_disabled'), on='name')
#    .merge(add_suffix(compile_only, '_compile_only'), on='name')
    .merge(add_suffix(jit, '_jit'), on='name')
)[['name', 'time_secs_native', 'time_secs_stock','time_secs_jit']] #  'time_secs_jit_disabled', 'time_secs_compile_only', 
combined['speedup_jit_stock'] = combined.time_secs_stock / combined.time_secs_jit
combined['speedup_native_jit'] = combined.time_secs_jit / combined.time_secs_native
combined = combined.sort_values(by='speedup_jit_stock', ascending=False).reset_index(drop=True)
print(combined.to_string())

                                         name  time_secs_native  time_secs_stock  time_secs_jit  speedup_jit_stock  speedup_native_jit
0                  chameneos_redux_lwt.600000          0.832997         7.115391       3.099997           2.295290            3.721498
1               thread_ring_lwt_stream.20_000          4.894969        28.747959      13.762132           2.088918            2.811485
2                                  kb_no_exc.          1.012073         6.334980       3.572056           1.773483            3.529445
3                             binarytrees5.21          5.227178        19.891920      11.249499           1.768249            2.152117
4                                knucleotide.         18.665458       119.142926      70.959650           1.679024            3.801656
5                    yojson_ydump.sample.json          0.319762         0.915832       0.569438           1.608309            1.780818
6                  test_decompress.64_524_288          

In [10]:
combined['speedup_jit_stock'].mean()

1.346423291055871